[原版（英文）图书地址](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/)


**代码修改和整理**：[黄海广](https://github.com/fengdu78)，原文修改成jupyter notebook格式，并增加和修改了部分代码，测试全部通过，所有数据集已经放在[百度云](data/README.md)下载。

**备注**：本章还在更新中，请等待

# 九、回到特征：将它们放到一起